In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

In [24]:
model_name = "gemini-1.5-flash-latest" 
google_api_key = "write_your_key_here"

In [25]:
def create_retriever(paths):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key=google_api_key)
    docments = []
    for path in paths:
        loader = PyPDFLoader(path)
        docments.extend(loader.load())
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=1500)
        splited_documents = text_splitter.split_documents(docments)
        vectorDB = FAISS.from_documents(splited_documents, embeddings)
    return vectorDB

In [26]:
course_retriever =  create_retriever([r"pdf_source\Lec_2.pdf"])
agent_retriever =  create_retriever([r"pdf_source\agent_info.pdf"])
user_retriever =  create_retriever([r"pdf_source\user_info.pdf"])

In [27]:
def course_info_retriever(question):
    docs = ""
    for result in course_retriever.similarity_search_with_relevance_scores(query=question):
      docs += result[0].page_content
    return docs  
    
def agent_info_retriever(question):
    docs = ""
    for result in agent_retriever.similarity_search_with_relevance_scores(query=question):
        docs += result[0].page_content
    return docs 
    
def user_info_retriever(question):
    docs = ""
    for result in user_retriever.similarity_search_with_relevance_scores(query=question):
        docs += result[0].page_content
    return docs 

In [28]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, output an answer using only the context retrieved from actions. Do not invent or explain answers; make sure your response is directly related to the question and sounds natural.
Use 'Thought' to describe your reflections on the question you’ve been asked, and Make sure of the pronouns if I am asking about you, or I am asking about myself, or I am asking about.
e.g.question: can you see
e.g. Thought: I need to understand what the user is asking. I'll use agent_info_retriever to get information about my capabilities.

Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

agent_info_retriever:
e.g. agent_info_retriever: when the question pertains to details about the chatbot or agent. For example, "When referring to 'you' or 'chatbot,' use agent_info_retriever.
returns agent_info_retriever.

course_info_retriever:
e.g. course_info_retriever: when i ask about in other domain or field.
returns course_info_retriever.

user_info_retriever:
e.g. user_info_retriever: just when i ask about user's personal details or any information about user.
returns user_info_retriever

Example session:

Question: hi, what is your name?
Thought: I should retrieve information from the agent_info_retrievert using agent_info_retriever.
Action: agent_info_retriever: question.
PAUSE

You will be called again with this:

Observation: Name: x-bot

You then output:

Answer: My name is x-bot
""".strip()

In [29]:
class SimpleAgent:
    def __init__(self, system='',):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
   
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=google_api_key)
        return llm.invoke(self.messages).content

    def query(self, question, known_actions, action_re, max_turns=5):  
        i = 0
        next_prompt = question
        
        while i < max_turns:
            i += 1
            result = self(next_prompt)
            print(result)
            
            actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
            
            if actions:
                action, action_input = actions[0].groups()
                if action not in known_actions:
                    raise Exception(f"Unknown action: {action}: {action_input}")
                print(f" -- running {action} {action_input}")
                observation = known_actions[action](action_input)
                print("Observation:", observation)
                next_prompt = f"Observation: {observation}"
            else:
                return

action_re = re.compile(r'^Action: (\w+): (.*)$')
known_actions = {
    "course_info_retriever": course_info_retriever,
    "agent_info_retriever": agent_info_retriever,
    "user_info_retriever": user_info_retriever
}

agent = SimpleAgent(system=prompt)
agent.query("what is my Faculty ", max_turns=10, known_actions=known_actions, action_re=action_re)

Thought: I need to retrieve information about the user's faculty. I will use the user_info_retriever.
Action: user_info_retriever: question.
PAUSE 

 -- running user_info_retriever question.
Observation: User information:  
Name:  Hossam  Eldin Tammam  Gad  
Age: 25  
Grad: 3.7 
Grad: A 
Faculty : Computer and information Technology  
Specialist  : artificial  intelligence
Answer: Your Faculty is Computer and information Technology. 

